# Textual Analysis Project

In this project, you will calculate disclosure tone for firms' earnings announcement 8-Ks in 2019 Q1. You will then calculate cumulative abnormal returns for the [0,+1] window surrounding the earnings announcement date and test the relation between earnings announcement tone and cumulative abnormal returns.

# Instructions

#### Import Modules

1. Import the following modules:
    1. requests
    2. pandas as pd
    3. word_tokenize from nltk.tokenize
    4. Counter from collections
    5. statsmodels.api as sm 
    6. html_to_text from MyFunctions (you will need to download the **MyFunctions.py** file and save it in your current working directory)    

#### Solution - Import Modules

In [ ]:
import requests
import pandas as pd
from nltk.tokenize import word_tokenize
from collections import Counter
import statsmodels.api as sm
from MyFunctions import html_to_text

#### Import Negative and Positive Word Lists

1. Import the negative word list into a **pandas** DataFrame called **neg_words** using the **pd.read_excel** function.
2. Import the postive word list into a **pandas** DataFrame called **pos_words** using the **pd.read_excel** function.
3. Rename the column header in the **neg_words** and **pos_words** DataFrames to 'token'.
3. Convert the words in the **neg_words** and **pos_words** DataFrames into lower case.

The word lists included in the **LoughranMcDonald_SentimentWordLists** Excel file available at https://sraf.nd.edu/textual-analysis/resources/. 

#### Solution - Import Negative and Positive Word Lists

In [ ]:
neg_words = pd.read_excel('LoughranMcDonald_SentimentWordLists_2018.xlsx', sheet_name='Negative', header=None)
neg_words = neg_words.rename(columns={0: "token"})
neg_words['token'] = neg_words['token'].str.lower()

pos_words = pd.read_excel('LoughranMcDonald_SentimentWordLists_2018.xlsx', sheet_name='Positive', header=None)
pos_words = pos_words.rename(columns={0: "token"})
pos_words['token'] = pos_words['token'].str.lower()

#### Create a Function to Obtain Tone

Create a new function called **get_tone** which takes the URL of an EDGAR filing and returns the **net_tone** of the filing, where **net_tone** is calculated as follows:

$
\begin{align}
NET\ TONE = \frac{\#\ POS\ WORDS - \#\ NEG\ WORDS}{\#\ POS\ WORDS + \#\ NEG\ WORDS}
\end{align}
$

See the **Disclosure Tone** module for additional instruction on how to create this function.

#### Solution - Create a Function to Obtain Tone

In [ ]:
def get_tone(url):
    
    # Obtain the text of the disclosure

    headers = {'User-Agent': 'ORGANIZATION youremail@yourinstitution.edu'} # AUG 2021 UPDATE -- YOU HAVE TO DECLARE A HEADER TO ACCESS THE EDGAR WEBSITE
    disclosure = requests.get(url,header=headers).text
    text = html_to_text(disclosure)
    
    # Convert text to lower case
    text = text.lower()
    
    # Create tokens
    tokens = word_tokenize(text)
    
    # Count tokens
    counts = Counter(tokens)

    # Create a DataFrame of our token counts
    df = pd.DataFrame.from_dict(counts, orient='index').reset_index()
    df = df.rename(columns={"index": "token", 0: "count"})
    df = df.sort_values(by=["count"],ascending=[False])
    
    # Merge with positive dictionary
    data = pd.merge(df, pos_words, on='token', how='left', indicator=True)
    data['pos'] = 0
    data.loc[data._merge == 'both', 'pos'] = 1
    data = data.drop(columns=['_merge'])
    
    # Merge with negative dictionary
    data = pd.merge(data, neg_words, on='token', how='left', indicator=True)
    data['neg'] = 0
    data.loc[data._merge == 'both', 'neg'] = 1
    data = data.drop(columns=['_merge'])

    # Calculate number of positive and negative tokens
    num_neg = data[data.neg == 1]['count'].sum()
    num_pos = data[data.pos == 1]['count'].sum()
    
    # Calculate tone
    net_tone = (num_pos - num_neg)/(num_pos + num_neg)
    
    return net_tone

#### Import Earnings Announcement 8-K URLs

Import the '8-K URLs.txt' file into a **pandas** DataFrame called **urls** using the **pd.read_csv** function. Note: The columns are delimited with the '|' symbol.

#### Solution - Import Earnings Announcement 8-K URLs

In [ ]:
urls = pd.read_csv('8-K URLs.txt',sep='|')
urls.head()

#### Compute Tone

1. Create a new file called 'tone.txt' in your current working directory using the **open** function. Write a new header containing the following variables delimited with the '|' symbol: **cik**, **ticker**, **date_filed**, **url**, and **net_tone**.
2. Loop through the filings in the **urls** DataFrame using the **iterrows** function and compute the **net_tone** of each filing using the **get_tone** function. For each filing, write a new row to the 'tone.txt' file containing the following variables delimited with a '|' symbol: **cik**, **ticker**, **date_filed**, **url**, and **net_tone**.

#### Solution - Compute Tone

In [ ]:
file1 = open('tone.txt','w')
file1.write('cik|ticker|date_filed|url|net_tone\n')
file1.close()

In [ ]:
dones1 = open('tone.txt').readlines()
dones = {}
for done in dones1:
    try:
        cik = done.split('|')[0]
        date_filed = done.split('|')[2]
        dones[cik+'|'+date_filed]=1
    except Exception:
        pass

for index,row in urls.iterrows():
    cik = str(row['cik'])
    ticker = row['ticker']
    date_filed = row['date_filed']
    url = row['URL']
    
    try:
        done = dones[cik+'|'+date_filed]
    except Exception:
        done = 0
        
    if done == 0:
        try:
            net_tone = get_tone(url)
            file1 = open('tone.txt','a')
            file1.write(cik+'|'+ticker+'|'+date_filed+'|'+url+'|'+str(net_tone)+'\n')
            file1.close()
        except Exception:
            print(cik)

#### Import Tone Data

Import the 'tone.txt' file to a new **pandas** DataFrame called **data** using the **pd.read_csv** function. Use the **parse_dates** option to read in the **date_filed** variable as a **datetime** object.

#### Solution - Import Tone Data

In [ ]:
data = pd.read_csv('tone.txt', sep='|', parse_dates=['date_filed'])
data.head()

#### Calculate Cumulative Abnormal Returns

1. Import the 'ret.csv' file to a new **pandas** DataFrame called **ret** using the **pd.read_csv** function. Use the **parse_dates** option to read in the **Date** variable as a datetime object. The 'ret.csv' file was created by scraping Yahoo! Finance (see the Web Scraping Yahoo! Finance Tutorials) and contains the following variables:
    1. **ticker**
    2. **Date**
    3. **ret** - Raw one-day return
    4. **ewret** - Equal-weighted index return
2. Calculate **abnret** as **ret** - **ewret**.
3. Sort the **ret** DataFrame by **ticker** (ascending order) and **Date** (descending order) using the **sort_values** function and the **ascending** option.
4. Use the **groupby** and **shift** functions to create a new variable called **abnret_lead1** equal to the return for each ticker on the subsequent trading date (i.e., date *t+1*). 
5. Create a new variable called **car01** equal to the cumulative abnormal return from day 0 to day +1 using the following formula:

    $
    \begin{align}
    CAR\ [0,+1]\ =\ [\ (1 + r_{t})\ \times\ (1 + r_{t+1})\ ] - 1
    \end{align}
    $

    where $r$ is the abnornal return on day t (current date) and day t+1 (subsequent trading date).

#### Solution - Calculate Cumulative Abnormal Returns

In [ ]:
# 1 - Import data

ret = pd.read_csv('ret.csv', parse_dates=['Date'])
ret.head()

In [ ]:
# 2 - Calculate abnret

ret['abnret'] = ret['ret'] - ret['ewret']
ret.head()

In [ ]:
# 3 - Sort data

ret = ret.sort_values(by=['ticker','Date'], ascending=[True,False])

In [ ]:
# 4 - Calculate abnret_lead1

ret['abnret_lead1'] = ret.groupby('ticker')['abnret'].shift(1)
ret.head()

In [ ]:
# 5 - Calculate car01

ret['car01'] = ((ret['abnret']+1) * (ret['abnret_lead1']+1)) - 1
ret.head()

#### Merge the Data and Ret DataFrames

1. Merge the **data** DataFrame with the **ret** DataFrame and call the resulting DataFrame **data**. Merge on **ticker** and **date_filed** in the **data** DataFrame and on **ticker** and **Date** in the **ret** DataFrame. Use an inner merge.
2. Drop all N/A values in the **data** DataFrame using the **dropna** function.

#### Solution - Merge the Data and Ret DataFrames

In [ ]:
data = pd.merge(data, ret, left_on=['ticker','date_filed'], right_on=['ticker','Date'], how='inner')
data = data.dropna()
data.head()

#### Run a Basic OLS Regression of Cumulative Abnormal Returns on Tone

1. Examine the summary statistics of the **net_tone** and **car01** variables using the **describe** function. What is the median value of **net_tone**?
2. Use the **OLS** function from the **statsmodels.api** module to run a basic OLS regression with the **car01** as the dependent variable and **net_tone** as the independent variable. Include a constant term in the regression. Print the OLS model summary output using the **summary** function.
3. Comment on the relation between tone and the cumulative abnormal return.

#### Solution - Run a Basic OLS Regression of Cumulative Abnormal Returns on Tone

In [ ]:
# 1 Summary Statistics

data[['net_tone','car01']].describe()

In [ ]:
# 2 OLS Regression

X = data["net_tone"]
y = data["car01"]
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
model.summary()

In [ ]:
# 3

# There is a positive and statistically significant relation between net_tone and car01.